In [1]:
import pandas as pd 
import numpy as np
import glob

In [2]:
def get_dataset(path,file_name,var):
    
    
    season_order = ['Winter', 'Pre-Monsoon', 'Monsoon', 'Post-Monsoon']
    
    df = pd.read_csv(path)
    
    # insert season column 
    df.insert(0, 'Season', season_order)
    
    # Melt the dataframe into Season,Year and Variable Name
    df = pd.melt(df, id_vars=['Season'], var_name='Year', value_name=var)
    df['Season'] = pd.Categorical(df['Season'], categories=season_order, ordered=True)
    df = df.sort_values(['Season', 'Year']).reset_index(drop=True)
    df.insert(0,'City',file_name)
    df = df.sort_values('City')
    return df

In [4]:
file_path = glob.glob('./*')[:-2]
file_path

['./Modis AOD_seasonal_data',
 './PM2p5_seasonal_data',
 './BLH_seasonal_data',
 './Insat AOD_seasonal_data']

In [5]:
dfs = []

# list to final dataframe of each parameter i.e INSAT AOD, MODIS AOD, PM2p5, BLH
df_para = []

for files in file_path:
    dfs = []
    variable_name = files.split('/')[-1].split('_')[0]
    
    # extract file from each parameter folder
    cities_list = glob.glob(files+'/*.csv')
   
    for city in cities_list:
        city_name = city.split('/')[-1].split('.')[0]
        df = get_dataset(city,city_name,variable_name)
        dfs.append(df)
    df_concat = pd.concat(dfs)
    df_concat = df_concat.sort_values('City')
    
    df_para.append(df_concat)

In [6]:
df_para

[           City        Season  Year  Modis AOD
 13  Bihar_Patna   Pre-Monsoon  2021   0.872333
 25  Bihar_Patna  Post-Monsoon  2019   0.928000
 24  Bihar_Patna  Post-Monsoon  2018   0.712500
 23  Bihar_Patna  Post-Monsoon  2017   0.719500
 22  Bihar_Patna  Post-Monsoon  2016   0.731000
 ..          ...           ...   ...        ...
 24   UP_Lucknow  Post-Monsoon  2018   0.680000
 25   UP_Lucknow  Post-Monsoon  2019   0.934500
 0    UP_Lucknow        Winter  2015   0.939333
 14   UP_Lucknow       Monsoon  2015   0.458250
 26   UP_Lucknow  Post-Monsoon  2020   0.784500
 
 [336 rows x 4 columns],
            City        Season  Year       PM2p5
 13  Bihar_Patna   Pre-Monsoon  2021   54.790984
 25  Bihar_Patna  Post-Monsoon  2019  118.279762
 24  Bihar_Patna  Post-Monsoon  2018  103.554645
 23  Bihar_Patna  Post-Monsoon  2017  130.846491
 22  Bihar_Patna  Post-Monsoon  2016  127.798611
 ..          ...           ...   ...         ...
 24   UP_Lucknow  Post-Monsoon  2018  115.422935
 25  

In [7]:
# Merge the first two dataframes
merged_df = pd.merge(df_para[0], df_para[1], on=['City', 'Season', 'Year'], how='outer')

# Merge the third dataframe
merged_df = pd.merge(merged_df, df_para[2], on=['City', 'Season', 'Year'], how='outer')

# Merge the fourth dataframe
merged_df = pd.merge(merged_df, df_para[3], on=['City', 'Season', 'Year'], how='outer')

# Sort the merged dataframe by City, Season, and Year columns
merged_df = merged_df.sort_values(by=['City', 'Season', 'Year'])

# Reorder the columns
merged_df = merged_df[['City', 'Season', 'Year', 'Modis AOD','Insat AOD','BLH', 'PM2p5']]


In [8]:
merged_df

,City,Season,Year,Modis AOD,Insat AOD,BLH,PM2p5
24,Bihar_Patna,Winter,2015,1.042000,0.250223,974.9108,NaN
26,Bihar_Patna,Winter,2016,1.250667,0.185159,1250.5979,166.870381
27,Bihar_Patna,Winter,2017,0.801333,0.168022,1112.5720,153.334547
23,Bihar_Patna,Winter,2018,0.991333,0.821012,1125.3460,162.460784
22,Bihar_Patna,Winter,2019,0.844333,0.733033,1129.2333,144.468391
...,...,...,...,...,...,...,...
330,UP_Lucknow,Post-Monsoon,2017,0.850000,0.607114,1417.4482,167.764973
331,UP_Lucknow,Post-Monsoon,2018,0.680000,0.548731,1835.0261,115.422935
332,UP_Lucknow,Post-Monsoon,2019,0.934500,0.813278,1228.4349,117.727001
335,UP_Lucknow,Post-Monsoon,2020,0.784500,0.674001,1519.3025,107.147698


In [9]:
merged_df.describe()

,Modis AOD,Insat AOD,BLH,PM2p5
count,320.000000,335.000000,336.000000,307.000000
mean,0.572360,1.137243,1347.786354,70.589994
std,0.229100,1.141232,569.137495,45.383944
min,0.155000,0.099633,188.043080,8.083705
25%,0.401000,0.273872,971.385922,37.749254
50%,0.527750,0.716424,1243.731700,54.790984
75%,0.761813,1.566922,1745.395825,99.480157
max,1.251000,4.665590,2924.515400,327.505618


In [10]:
print(glob.glob('./Datasets/Train.csv'))

['./Datasets/Train.csv']


In [18]:
#merged_df.to_csv('./Datasets/Train.csv', index=False)

In [12]:
merged_df.tail()

,City,Season,Year,Modis AOD,Insat AOD,BLH,PM2p5
330,UP_Lucknow,Post-Monsoon,2017,0.8500,0.607114,1417.4482,167.764973
331,UP_Lucknow,Post-Monsoon,2018,0.6800,0.548731,1835.0261,115.422935
332,UP_Lucknow,Post-Monsoon,2019,0.9345,0.813278,1228.4349,117.727001
335,UP_Lucknow,Post-Monsoon,2020,0.7845,0.674001,1519.3025,107.147698
308,UP_Lucknow,Post-Monsoon,2021,0.6975,3.200981,1137.1241,74.719012


In [13]:
len(df_para[1].City.unique())

12

In [14]:
len(merged_df.City.unique())

12

In [16]:
len(merged_df.City.unique())

12